## 基于EPIC模型的土壤可蚀性因子计算

In [2]:
import geemap
import ee

Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
import os
os.chdir("F:\论文写作\欣雨学姐\研究区")

In [4]:
shp = geemap.shp_to_ee("青藏高原范围.shp")
roi = shp.geometry()
Map.center_object(roi, 7)
Map.addLayer(roi, {}, 'roi')

In [5]:
clay = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").clip(roi).toArray().arrayReduce('mean', [0]).arrayGet([0])

sand = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").clip(roi).toArray().arrayReduce('mean', [0]).arrayGet([0])

silt = sand.expression("100 - sand - clay", {'sand': sand, 'clay': clay})

Map.addLayer(clay, {'min': 0, 'max': 100}, 'clay')
Map.addLayer(sand, {'min': 0, 'max': 100}, 'sand')
Map.addLayer(silt, {'min': 0, 'max': 100}, 'silt')

In [6]:
orgC = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").clip(roi).toArray().arrayReduce('mean', [0]).arrayGet([0]).multiply(0.02)
Map.addLayer(orgC, {'min': 0, 'max': 2.4}, 'orgC')

In [7]:
Kepic = orgC.expression("(0.2 + 0.3 * exp(-0.0256 * sand * (1 - silt / 100))) * " + 
                        "(silt / (clay + silt)) ** 0.3 * " + 
                        "(1 - 0.25 * orgC / (orgC + exp(3.72 - 2.95 * orgC))) * " + 
                        "(1 - 0.7 * (1 - sand / 100)/((1 - sand / 100) + exp(-5.51 + 22.9 * (1 - sand / 100))))", 
                        {
                            'sand': sand,
                            'clay': clay,
                            'silt': silt,
                            'orgC': orgC
                        }
                        )
Map.addLayer(Kepic, {}, 'Kepic')

In [8]:
Map.addLayer(Kepic, {}, 'Kepic')

In [10]:
K = Kepic.expression("0.1317 * (-0.01383 + 0.51575 * Kepic)", {'Kepic': Kepic})
#Map.addLayer(K, {'min': 0, 'max': 0.1}, 'K')

In [11]:
geemap.download_ee_image(K, 'K.tif', roi, 'epsg:3857', scale=1000)

K.tif: |          | 0.00/51.2M (raw) [  0.0%] in 00:00 (eta:     ?)

Tile downloaded failed, retry 1 of 5.  URL: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/13dff0fb75a3cdc79ba8eaf280542a05-75411c848ee0131de540b19eeba7f01c:getPixels. ('Connection broken: IncompleteRead(1358823 bytes read, 6796157 more expected)', IncompleteRead(1358823 bytes read, 6796157 more expected)).
There is no STAC entry for: None
